# Introduction

Kita akan menggunakan [Bank Marketing Dataset](https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset) dari Kaggle. 

In [ ]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


#plt.style.use('seaborn')

# Data Preprocessing


## Loading data

Dokumentasi dataset dapat dilihat [di sini](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing) 

Deskripsi kolom adalah sebagai berikut:

Variabel input :
1. age (numeric)
2. job : type of job (categorical: `'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown'`)
3. marital : marital status (categorical: `'divorced','married','single','unknown'`; note: `'divorced'` means divorced or widowed)
4. education (categorical: `'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown'`)
5. default: has credit in default? (categorical: `'no','yes','unknown'`)
6. housing: has housing loan? (categorical: `'no','yes','unknown'`)
7. loan: has personal loan? (categorical: `'no','yes','unknown'`)
8. contact: contact communication type (categorical: `'cellular','telephone'`)
9. month: last contact month of year (categorical: `'jan', 'feb', 'mar', ..., 'nov', 'dec'`)
10. day_of_week: last contact day of the week (categorical: `'mon','tue','wed','thu','fri'`)
11. duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
12. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. previous: number of contacts performed before this campaign and for this client (numeric)
15. poutcome: outcome of the previous marketing campaign (categorical: `'failure','nonexistent','success'`)

Target :
21. deposit. has the client subscribed a term deposit? (binary: 'yes','no')

In [ ]:
df_bank = pd.read_csv('https://raw.githubusercontent.com/urfie/DataAnalytics/main/bank.csv')

print('Dataframe shape:', df_bank.shape)

In [ ]:
df_bank.head()

##Data Cleansing

Menurut keterangan di atas, kolom durasi baru akan diketahui setelah panggilan dilakukan, yaitu setelah nilai target diketahui. Hal ini dapat digolongkan sebagai 'data leakage' yang akan mempengaruhi kualitas prediktor yang dihasilkan. Untuk itu kita perlu membuang kolom ini dari daftar fitur kita.

In [ ]:
df_bank = df_bank.drop('duration', axis=1)

print('Dataframe shape:', df_bank.shape)
df_bank.head()

### Cek nilai kosong/hilang

Untuk mengecek apakah ada nilai/kolom yang kosong, kita gunakan perintah `isnull()` diikuti dengan `sum()`


In [ ]:
df_bank.isnull().sum()

Dari hasil di atas terlihat tidak ada kolom yang nilainya kosong/null/hilang. 

Jika ditemukan nilai kosong/null/hilang, ada beberapa pilihan yang dapat dilakukan : 
1. Hapus row dengan nilai null, atau
2. Lakukan imputasi untuk nilai-nilai null tersebut. Nilai yang digunakan untuk imputasi dapat berupa nilai default, rata-rata, mode, dll.

###Cek nilai unique tiap kolom

Untuk kolom-kolom bertipe kategorik, kita bisa melakukan checking terhadap nilai unik-nya, untuk melihat apakah ada nilai invalid/tidak standar yang perlu kita perbaiki.

In [ ]:
#list kolom numerik dan kategorik
num_cols = ['age','balance','campaign','pdays','previous']
cat_cols = ['job','marital','education','default','housing','loan','contact','day','month','poutcome']

#cek nilai di kolom bertipe kategorik
for i in cat_cols:
  print(df_bank[i].unique())

##Eksplorasi Data

### Distribusi target 

Salah satu hal yang penting diketahui adalah bagaimana distribusi target variabel, apakah dataset yang kita proses seimbang atau tidak.

In [ ]:
df_bank['deposit'].value_counts().plot.barh()

Ternyata jumlah `yes` dan `no` dalam dataset kita seimbang. 

Untuk data yang tidak seimbang, ada beberapa metode yang bisa dilakukan, diantaranya 
1. Memilih metrik yang tepat untuk evaluasinya
2. Melakukan resampling : under/oversampling, resampling dengan rasio berbeda
3. Menggunakan K-fold cross validation
4. Menggunakan metode ensemble
5. Melakukan klastering pada kelas yang jauh lebih besar, dan mengambil medoid-nya saja
6. dll.

###Distribusi kolom input

Selanjutnya kita bisa melihat distribusi kolom input terhadap nilai target.

In [ ]:
def plot_chart(xcol, ycol, dataframe):
    ax=sns.countplot(x=xcol, hue=ycol, data=dataframe)
    plt.xticks(rotation=90)
    

In [ ]:
plot_chart('job',"deposit",df_bank)

In [ ]:
plot_chart('marital',"deposit",df_bank)

In [ ]:
plot_chart('education',"deposit",df_bank)

In [ ]:
plot_chart('default',"deposit",df_bank)

In [ ]:
plot_chart('housing',"deposit",df_bank)

In [ ]:
plot_chart('loan',"deposit",df_bank)

In [ ]:
plot_chart('contact',"deposit",df_bank)

###Hubungan antar variabel numerik

Untuk melihat hubungan antar variabel numerik, kita bisa menghitung korelasi dan memvisualisasikannya dengan heatmap.

In [ ]:
#compute correlation matrix
corr = df_bank[num_cols].corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="RdBu_r",annot=True)
#plt.title('Correlation between Numeric Variables')

##Transformasi variabel kategorik

Scikit learn memerlukan input berupa array numerik, sehingga kita perlu mengubah variabel-variabel bertipe kategorik menjadi numerik dengan melakukan one-hot encoding.



###Transform input features

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore') 
cat_cols = ['job','marital','education','default','housing','loan','contact','day','month','poutcome']

In [ ]:
# contoh one hot encoding untuk kolom marital status
encoded = enc.fit_transform(df_bank[['marital']]).toarray() #encode dan ubah menjadi array

print(df_bank.marital.unique())
print('\nBentuk Matrix setelah melakukan encoding 1 variable')
print(encoded.shape)
print('\nNilai sebelum encoding (3 baris terakhir)')
print(df_bank[['marital']].tail(3))
print('\nNilai setelah encoding (3 baris terakhir)')
print(encoded[-3:, :]) #

Ubah seluruh variabel kategorik : 

In [ ]:
cat_cols.remove('marital')

for col in cat_cols:
  encoded = np.concatenate([encoded, enc.fit_transform(df_bank[[col]]).toarray()], axis = 1) 

print("Matriks fitur setelah encoding seluruh variabel kategorik:")
print(encoded.shape)

print("\nDua baris pertama:")
print(encoded[:2, :])

Menggabungkan dengan variabel-variabel numerik

In [ ]:
num_cols = ['age','balance','campaign','pdays']#,'previous']

numeric_columns = np.array(df_bank[num_cols])

encoded = np.concatenate([encoded, numeric_columns], axis = 1)

print("Matriks fitur setelah menggabungkan seluruh variabel:")
print(encoded.shape)
print("\nDua baris pertama:")
print(encoded[:2, :])

###Transform target

Kita gunakan `labelBinarizer` untuk mengubah kolom target dari `'yes'/'no'` menjadi `1/0`

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
encoded_target = lb.fit_transform(df_bank[['deposit']]).ravel()

encoded_target[:3]

##Split dataset ke dalam train-test set



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoded , encoded_target, 
                                                    shuffle = True, 
                                                    test_size=0.2, 
                                                    random_state=1)

# Show the Training and Testing Data
print('Dimensi training feature:', X_train.shape)
print('Dimensi testing feature:', X_test.shape)
print('Dimensi training target:', y_train.shape)
print('Dimensi training target:', y_test.shape)

##Penskalaan variabel numerik

Variabel numerik perlu diskalakan ulang sehingga memiliki rentang nilai yang serupa, agar perbedaan dalam rentang nilai tersebut tidak memengaruhi model training.

In [ ]:
#variabel numerik yang memiliki rentang berbeda-beda
df_bank[num_cols].head(5)

Variabel numerik ada di kolom 75 (base 0) dan seterusnya, sehingga kita akan melakukan scaling terhadap `X_train` maupun `X_test` mulai kolom ke 75.

Karena Regresi Logistik memerlukan asumsi data terdistribusi normal, maka kita akan menggunakan `StandardScaler` untuk melakukan standarisasi data, yaitu mengeset mean ke 0 dan standard deviasi 1. 

Perhatikan bahwa scaler hanya di-fit pada data training, sedangkan data test diskalakan menggunakan parameter dari data training. Hal ini dilakukan untuk menghindari data leakage pada test set.

In [ ]:

scaler = StandardScaler().fit(X_train[:,75:]) 

X_train[:,75:] = scaler.transform(X_train[:,75:])
X_test[:,75:] = scaler.transform(X_test[:,75:])


print("Dimensi data training sesudah dilakukan penskalaan :\n", X_train.shape)
print("Dimensi data test sesudah dilakukan penskalaan :\n", X_test.shape)

print("Dua baris pertama sesudah dilakukan penskalaan :\n", X_train[:2,:])

##Melatih model : Regresi Logistik

Kita akan menggunakan model regresi logistik untuk melakukan prediksi terhadap apakah seorang nasabah akan melakukan deposit. Regresi logistik ada dalam package `linear_model`

In [ ]:
from sklearn import linear_model

lm = linear_model.LogisticRegression(max_iter=1000)
model = lm.fit(X_train, y_train)

##Evaluasi Model

Sebelum melakukan evaluasi, kita perlu menentukan dulu metriks apa yang paling relevan untuk problem yang akan kita pecahkan.

Untuk kasus ini, kita tertarik pada nasabah yang benar-benar melakukan deposit (Actual Positive), karena kegagalan memprediksi nasabah yang melakukan deposit (False Negative) dapat dilihat sebagai 'lost opportunity'. 

Untuk itu kita akan fokus pada berapa banyak data positif yang berhasil diprediksi (*True Positive*), dibanding seluruh data positif sebenarnya (*True Positive + False Negative*).

<br><div align="center">
$\frac{TP}{TP + FN} = Recall = True Positive Rate$
</div><br>





In [ ]:
from sklearn import metrics

# Lakukan prediksi terhadap test set
y_pred = model.predict(X_test)

# Hitung akurasi, presisi, recall, dan f1-score
print('Akurasi:', metrics.accuracy_score(y_test, y_pred))
print('Presisi:', metrics.precision_score(y_test, y_pred))
print('Recall:', metrics.recall_score(y_test, y_pred))
print('F1 Score:', metrics.f1_score(y_test, y_pred))


In [ ]:
# Display confusion matrix
cm = metrics.confusion_matrix(y_test, y_pred)
from sklearn.metrics import ConfusionMatrixDisplay
print('Confusion Matrix:\n', cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
plt.show()

Plot kurva ROC untuk melihat kinerja model pada tiap-tiap threshold, sebagai fungsi FPR-TPR

In [ ]:
#Prediksi kelas dan nilai probabilitas tiap kelas
y_proba = model.predict_proba(X_test)

fpr, tpr, thresh = metrics.roc_curve(y_test, y_proba[:, 1])
roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                   estimator_name='Logistic Regression')
display.plot()
plt.plot([0, 1], [0, 1], color = 'g')
plt.show()